In [139]:
import geopandas as gp
from shapely import wkt
from shapely.geometry import Point, Polygon
from shapely.ops import unary_union
import pandas as pd
import numpy as np
from pprint import pprint
import os
import glob
import openpyxl
import matplotlib.pyplot as plt
import matplotlib.ticker as mtick
import seaborn as sns
import plotly.express as px #if using plotly
import folium
import warnings
import uuid
from sklearn.preprocessing import StandardScaler

In [140]:
pd.set_option('display.max_columns', None)  # display all columns
pd.options.display.float_format = '{:20,.2f}'.format  # suppress scientific notation

### RQ #2) Which specific flares are the most impactful? 

### RQ #3) Which specific block grous are being most impacted? 


Exploratory question; no hypothesis testing

EJ index = (The Environmental Indicator Percentile for Block Group) x (Demographic Index for Block Group)
  
a) Determine flaring impact metric. 

Something like: buffer_population x EJ Index x flare volume  

b) Calculate impact per flare  

c) Visualize top_x flares by impact  


### Read in files

In [141]:
#ca_state = gp.read_file("data/CA_State_TIGER2016.shp")  # CA state
ca_counties = gp.read_file("data/CA_Counties_TIGER2016.shp")  # CA counties
ca_counties.rename(columns={'NAMELSAD':'cnty_name'}, inplace=True)  # old:new. Match col names for merging

In [142]:
#ca_bg = gp.read_file("data/ca_bg_clean.shp")  # CA block groups, cleaned
# CA block groups merged with EJscreen data
ca_bg_joined = gp.read_file("data/ca_bg_joined_clean.shp")

# update col name for correct area b/c it gets saved as a truncated version when saved as a shapefile out of the 
# main data cleaning notebook
ca_bg_joined.rename(columns={'shape_ar_1':'bg_area'}, inplace=True)  # old:new.
print(f'BGs found: {len(ca_bg_joined)}')

BGs found: 25590


In [143]:
# just CA flares
ca_flares = gp.read_file("data/ca_flares_clean.shp") 

print(f'Flares found: {len(ca_flares)}')

Flares found: 117


In [144]:
# set common crs for project
# epsg3310: https://epsg.io/3310-1739
# units: meters
meters_crs = 3310  # Projected crs. this should be good for this overlay() calculation and all of project. 

ca_counties = ca_counties.to_crs(meters_crs)
ca_flares = ca_flares.to_crs(meters_crs)
ca_bg_joined = ca_bg_joined.to_crs(meters_crs)

### Update ca_flares df

In [145]:
col_list = ['flare_id','flare_cate','cnty_name','BCM_avg','geometry']

subset = ca_flares[col_list].copy()

# assign the subsetted copy back to the original variable name to keep total # of df names small 
# throughout project
ca_flares = subset

In [146]:
ca_flares.sample(2)

,flare_id,flare_cate,cnty_name,BCM_avg,geometry
101,2d637c8e295983,Upstream,Santa Barbara County,0.00,POINT (-36516.426 -355944.880)
45,e7645bc6107104,Upstream,Kern County,0.00,POINT (94072.137 -332054.081)


### Update ca_bg_joined df

In [147]:
# subset to only cols needed
subset = ca_bg_joined[['bg_id', 'cnty_name', 'ACSTOTPOP', 'MINORPOP',
                           'D_PM25_2', 'bg_area', 'geometry']].copy()

# assign the subsetted copy back to the original variable name
ca_bg_joined = subset

ca_bg_joined.rename(columns={'geometry':'bg_geom'}, inplace=True)  # old:new. Match col names for merging

## Create "baseline" df for use in further BG-specific and flare-specific analysis

In [148]:
# set geometry columns
ca_bg_joined = ca_bg_joined.set_geometry('bg_geom')

# and rename for clarity
ca_flares.rename(columns={'geometry':'flarepts_geom'}, inplace=True)  # old:new. Match col names for merging
ca_flares = ca_flares.set_geometry('flarepts_geom')

In [149]:
len(ca_bg_joined['bg_id'].unique())

25590

In [256]:
buffer_size=100
#buffer_size=1000
#buffer_size=2000

In [257]:
buffer_col = f"{buffer_size}_buff_geom"
ca_flares[buffer_col] = ca_flares['flarepts_geom'].buffer(distance=buffer_size)

ca_flares = ca_flares.set_geometry(buffer_col)

# subset of desired columns from flares_df
flares_sjoin_subset = ca_flares[[buffer_col,'flare_id','flare_cate','BCM_avg']]

# Left join the flares_sjoin_subset cols to the bg_socialdf based on if any of the flares geometries intersect 
# with the bg geometries
intersect_baseline = gp.sjoin(ca_bg_joined, flares_sjoin_subset, how='left', predicate='intersects')

# Bring in buffer col that is not brought in by the sjoin() function for some unknown reason. Frustrating.
intersect_baseline = pd.merge(intersect_baseline, flares_sjoin_subset[['flare_id', buffer_col]], 
                             on='flare_id', how='left')

# set geom to the buffers and filter df down to just these BGs that have a flare_id associated with them. 
# There's no need to calculate any of the following for non-intersected BGs. 
intersect_baseline = intersect_baseline.set_geometry(buffer_col)
intersect_baseline = intersect_baseline[intersect_baseline.is_valid].copy()
print(f'{buffer_size}m BG-Buffer intersections: {len(intersect_baseline)}')

100m BG-Buffer intersections: 118


In [258]:
# Iterate through each bg_id and flare_id pairing and calculate 
# the proportion of the buffer that intersects with the BG geom, relative to the total buffer size.
# We need to know the proportion of each buffer in each bg geom in order to assign the percentage of
# flare volume to each bg in the case of incomplete overlap.

# First check for null or invalid geometries in buffer_2000m column
intersect_baseline['buffer_valid'] = intersect_baseline[buffer_col].is_valid

# Initialize intersect proportion "b_int_prop" and intersect area "b_int_area" columns with NaN values
intersect_baseline[f'{buffer_size}_buff_int_prop'] = np.nan
intersect_baseline[f'{buffer_size}_buff_int_area'] = np.nan

# Loop through each block group and flare pairing
for index, row in intersect_baseline.iterrows():
    # Check if buffer_2000m geometry is valid
    if row['buffer_valid']:
        # Calculate intersection area
        intersection = row[buffer_col].intersection(row['bg_geom'])  # prop of buffer in the bg geom
        intersection_area = intersection.area
        # Calculate buffer area
        buffer_area = row[buffer_col].area
        # Calculate intersection proportion
        buffer_int_prop = intersection_area / buffer_area
        # Update int_prop and int_area columns with calculated values
        intersect_baseline.at[index, f'{buffer_size}_buff_int_prop'] = buffer_int_prop
        intersect_baseline.at[index, f'{buffer_size}_buff_int_area'] = intersection_area
        intersect_baseline.at[index, f'{buffer_size}_buff_tot_area'] = buffer_area

    else:
        # Assign 0 to b_int_prop and b_int_area if buffer_2000m geometry is invalid
        intersect_baseline.at[index, f'{buffer_size}_buff_int_prop'] = 0
        intersect_baseline.at[index, f'{buffer_size}_buff_int_area'] = 0
        intersect_baseline.at[index, f'{buffer_size}_buff_tot_area'] = 0


# Drop buffer_valid column as it's no longer needed
intersect_baseline.drop('buffer_valid', axis=1, inplace=True)

In [259]:
# Now a similar analysis but this time we want to know the BG-buffer intersection proportion relative to 
# the total area of the block group, in order to use that proportion to assign counts of BG demographic
# variables to the intersection area. 

# Combine all flares into a single multipolygon unit
temp = ca_flares.unary_union
all_buffers = gp.GeoDataFrame({'geometry': [temp]}, crs=ca_flares.crs) # convert back to geodf for processing

intersect_baseline = intersect_baseline.set_geometry('bg_geom')

# Make sure our geom column is set correctly to "bg_geom" rather than the flare buffer
if intersect_baseline.geometry.name == "bg_geom":
    int_temp = gp.overlay(intersect_baseline, all_buffers, how='intersection')  

    int_temp.rename(columns={'geometry':f'{buffer_size}_bg_int_geom'}, inplace=True)  # old:new. For clarity

    # Set geometry to the intersections
    int_temp = int_temp.set_geometry(f'{buffer_size}_bg_int_geom')  

    # Create new 'area' column for the areas of the intersections
    int_temp[f'{buffer_size}_bg_int_area'] = int_temp.area

    # Calculate the proportion of each block group intersection relative to total block broup area
    int_temp[f'{buffer_size}_bg_int_prop'] = int_temp[f'{buffer_size}_bg_int_area'] / int_temp['bg_area']

    # Merge the new area and props variables
    intersect_baseline = pd.merge(intersect_baseline, 
                                 int_temp[['bg_id',
                                           f'{buffer_size}_bg_int_prop',
                                           f'{buffer_size}_bg_int_area', 
                                           f'{buffer_size}_bg_int_geom']], 
                                 on=['bg_id'], 
                                 how='left').fillna({f'{buffer_size}_bg_int_prop':0, 
                                                     f'{buffer_size}_bg_int_area':0})

else:
    print("Error: Geometry column is not named 'bg_geom'")

In [260]:
# Apply the proportion to each demographic variable to find counts by variable
# i.e. this answers "how many of x live in the X buffer radius intersection area?"
demo_vars = ['ACSTOTPOP', 'MINORPOP']
for var in demo_vars:
    intersect_baseline[f'{buffer_size}_{var}_bg_int_count'] = (intersect_baseline[var] *
                                                               intersect_baseline[f'{buffer_size}_bg_int_prop'])

# Find *overall* proportions for each demo var by dividing var count by respective total population
# i.e. this answers "what overall percentage of this block group is minority status?"
intersect_baseline['MINORPOP_bg_totprop'] = intersect_baseline['MINORPOP'] / intersect_baseline['ACSTOTPOP']

# calculate percentage of minorities in intersection for each BG
intersect_baseline[f'{buffer_size}_bg_int_mnr_prop'] = (intersect_baseline[f'{buffer_size}_MINORPOP_bg_int_count']
                                                        / intersect_baseline[f'{buffer_size}_ACSTOTPOP_bg_int_count'])

In [261]:
# There are a ton of duplicate rows getting introduced at some point of the process. Ideally I'll come back and 
# troubleshoot, but in the meantime I'm just removing these dupes. 

df_baseline_all = intersect_baseline.drop_duplicates(subset=['bg_id', 'flare_id']).copy()

# I think the sjoin() functions ocassionally do weird things to data types. 
# Changing year back to an integer
#df_baseline_all['year'] = df_baseline_all['year'].astype('int64')


In [262]:
len(df_baseline_all)

118

In [263]:
# grab two specific bgs to check
df_baseline_all[df_baseline_all['bg_id'].isin([60379800331, 60379800111])]


,bg_id,cnty_name,ACSTOTPOP,MINORPOP,D_PM25_2,bg_area,bg_geom,index_right,flare_id,flare_cate,BCM_avg,100_buff_geom,100_buff_int_prop,100_buff_int_area,100_buff_tot_area,100_bg_int_prop,100_bg_int_area,100_bg_int_geom,100_ACSTOTPOP_bg_int_count,100_MINORPOP_bg_int_count,MINORPOP_bg_totprop,100_bg_int_mnr_prop
96,60379800331,Los Angeles County,14,14,94.00,"28,637,875.64","POLYGON ((162259.677 -471740.729, 162351.365 -...",113.00,915ae204194312,Refinery,0.00,"POLYGON ((165297.511 -469609.028, 165297.030 -...",1.00,"31,365.48","31,365.48",0.00,"104,795.09","POLYGON ((165296.749 -469620.721, 165295.590 -...",0.05,0.05,1.00,1.00
101,60379800331,Los Angeles County,14,14,94.00,"28,637,875.64","POLYGON ((162259.677 -471740.729, 162351.365 -...",106.00,771e0f67682313,Refinery,0.00,"POLYGON ((165054.446 -469593.965, 165053.965 -...",1.00,"31,365.48","31,365.48",0.00,"104,795.09","POLYGON ((165296.749 -469620.721, 165295.590 -...",0.05,0.05,1.00,1.00
106,60379800331,Los Angeles County,14,14,94.00,"28,637,875.64","POLYGON ((162259.677 -471740.729, 162351.365 -...",107.00,68580388957987,Refinery,0.00,"POLYGON ((165234.921 -469587.135, 165234.440 -...",1.00,"31,365.48","31,365.48",0.00,"104,795.09","POLYGON ((165296.749 -469620.721, 165295.590 -...",0.05,0.05,1.00,1.00
111,60379800331,Los Angeles County,14,14,94.00,"28,637,875.64","POLYGON ((162259.677 -471740.729, 162351.365 -...",108.00,b91dbc75798361,Refinery,0.00,"POLYGON ((165383.806 -469521.668, 165383.324 -...",1.00,"31,365.48","31,365.48",0.00,"104,795.09","POLYGON ((165296.749 -469620.721, 165295.590 -...",0.05,0.05,1.00,1.00
116,60379800331,Los Angeles County,14,14,94.00,"28,637,875.64","POLYGON ((162259.677 -471740.729, 162351.365 -...",112.00,b5b74874510417,Refinery,0.00,"POLYGON ((165297.510 -469510.426, 165297.028 -...",1.00,"31,365.48","31,365.48",0.00,"104,795.09","POLYGON ((165296.749 -469620.721, 165295.590 -...",0.05,0.05,1.00,1.00


# Aggregate data by BG and export files for Tableau

take baseline dfs and create copies clearly defined for this BG analysis, separate from the work that is needed
for the flare buffer analysis below. Both analyses modify these same two dfs in different ways and I want 
to make sure I keep the calculations separated

#### Standardize variables

EJ index and total population in the intersection(s) are already encoded at the level of a BG, so they're ready to normalize. 

Flare volume needs to multiplied by the proportion of the flare's buffer zone that resides in a BG, and then all (in the case of multiple flares in a BG) adjusted flare volumes summed up to generate a variable for total flare volume a BG is exposed to.  

In [264]:
def bg_calc_adj_flare_vol(df, buffer_size):
    """
    Function for calculating total flare volume to be assigned to each BG, based on amount of overlap.
    
    "bcm_adj" = adjusted bcm value for a given flare_id and bg_id, based on the proportion of the buffer zone that 
    overlaps with a given bg_id. Equal to the bcm_avg * b_int_prop (the overlap)
    
    "bcm_adj_tot" = the sum of the bcm_adj values for a given bg_id
    """
    
    # calculate the product of bcm_avg and buff_int_prop for each flare and bg combination
    df[f'{buffer_size}_bcm_int_adj'] = df['BCM_avg'] * df[f'{buffer_size}_buff_int_prop']
    
    # sum the adjusted volumes for all flare and bg combinations
    adj_vol = df.groupby(['flare_id', 'bg_id'])[f'{buffer_size}_bcm_int_adj'].sum().reset_index()
    
    # sum those adjusted volumes for all flares in each block group
    bg_vol = adj_vol.groupby('bg_id')[f'{buffer_size}_bcm_int_adj'].sum().reset_index()
    
    # merge the total adjusted block group volumes with the original dataframe
    df = pd.merge(df, bg_vol, on='bg_id', how='left', suffixes=('', '_tot'))
    
    return df

In [265]:
df_bg_intersect = df_baseline_all.groupby('bg_id').apply(bg_calc_adj_flare_vol, buffer_size=buffer_size)

In [266]:
# Define the variables to standardize

#'bcm_adj_tot' = Sum of the adjusted flare volume(s) that intersect w the BG. 
#'D_PM25_2' = EJ index for the BG
#'ACSTOTPOP_int_count' = BG population that resides in the intersection. = BGtotpop * intersection_proportion

varlist = [f'{buffer_size}_bcm_int_adj_tot', 'D_PM25_2', f'{buffer_size}_ACSTOTPOP_bg_int_count']

# Create a StandardScaler object that will transform selected variables to have a mean of zero and 
# sd of 1.
scaler = StandardScaler()

# Fit and transform the selected variables
intersect_norm = scaler.fit_transform(df_bg_intersect[varlist])

# Create new variables with the standardized values
intersect_norm = pd.DataFrame(intersect_norm, columns=[var + '_n' for var in varlist])

intersect_norm = intersect_norm.reset_index(drop=True)
df_bg_intersect = df_bg_intersect.reset_index(drop=True)

# Concatenate the new variables with the original DataFrame
df_bg_intersect = pd.concat([df_bg_intersect, intersect_norm], axis=1)

In [267]:
# Create calculations of # of counties affected and counts of BGs according to different
# "bins" of overlap

df_bg_intersect[f'{buffer_size}_bg_int_25%orless'] = (df_bg_intersect.groupby('bg_id')
                                        [f'{buffer_size}_bg_int_prop'].apply(lambda x: (x <= .25).sum()).sum())

df_bg_intersect[f'{buffer_size}_bg_int_50%orless'] = (df_bg_intersect.groupby('bg_id')
                                        [f'{buffer_size}_bg_int_prop'].apply(lambda x: (x <= .5).sum()).sum())

df_bg_intersect[f'{buffer_size}_bg_int_75%orless'] = (df_bg_intersect.groupby('bg_id')
                                        [f'{buffer_size}_bg_int_prop'].apply(lambda x: (x <= .75).sum()).sum())

df_bg_intersect[f'{buffer_size}_bg_int_anyint'] = (df_bg_intersect.groupby('bg_id')
                                        [f'{buffer_size}_bg_int_prop'].apply(lambda x: (x <= 1).sum()).sum())



In [268]:
# counties affected
temp = gp.GeoDataFrame(df_bg_intersect[['bg_id', 'flare_id', f'{buffer_size}_bg_int_geom']], 
                                         geometry=f'{buffer_size}_bg_int_geom')

counties_intersect = gp.sjoin(ca_counties, temp, how='inner', predicate='contains')  

df_bg_intersect[f'{buffer_size}_num_counties'] = len(counties_intersect['cnty_name'].unique())

In [269]:
df_bg_intersect.sample(10)

,bg_id,cnty_name,ACSTOTPOP,MINORPOP,D_PM25_2,bg_area,bg_geom,index_right,flare_id,flare_cate,BCM_avg,100_buff_geom,100_buff_int_prop,100_buff_int_area,100_buff_tot_area,100_bg_int_prop,100_bg_int_area,100_bg_int_geom,100_ACSTOTPOP_bg_int_count,100_MINORPOP_bg_int_count,MINORPOP_bg_totprop,100_bg_int_mnr_prop,100_bcm_int_adj,100_bcm_int_adj_tot,100_bcm_int_adj_tot_n,D_PM25_2_n,100_ACSTOTPOP_bg_int_count_n,100_bg_int_25%orless,100_bg_int_50%orless,100_bg_int_75%orless,100_bg_int_anyint,100_num_counties
55,60375409024,Los Angeles County,0,0,0.00,"1,225,955.21","POLYGON ((158875.798 -455207.614, 158869.366 -...",105.00,55af1327142492,Upstream,0.00,"POLYGON ((159189.293 -455288.160, 159188.812 -...",1.00,"31,365.48","31,365.48",0.03,"31,365.48","POLYGON ((159189.293 -455288.160, 159188.812 -...",0.00,0.00,NaN,NaN,0.00,0.00,-1.12,-1.46,-0.35,118,118,118,118,11
65,60590626434,Orange County,594,252,19.52,"2,118,889.96","POLYGON ((201605.690 -485634.058, 201682.676 -...",86.00,f86bc4a0328919,Upstream,0.01,"POLYGON ((202229.924 -486745.806, 202229.443 -...",1.00,"31,365.48","31,365.48",0.02,"37,102.69","POLYGON ((202216.381 -486798.106, 202213.632 -...",10.40,4.41,0.42,0.42,0.01,0.02,1.26,-0.69,1.70,118,118,118,118,11
21,60290033042,Kern County,1040,279,31.94,"1,051,991,808.27","POLYGON ((7651.644 -285527.714, 7653.631 -2855...",49.00,354ae1c9634624,Upstream,0.00,"POLYGON ((47065.526 -304151.318, 47065.044 -30...",1.00,"31,365.48","31,365.48",0.00,"274,294.75","MULTIPOLYGON (((47097.528 -304252.983, 47097.0...",0.27,0.07,0.27,0.27,0.00,0.01,0.58,-0.21,-0.30,118,118,118,118,11
80,61110003041,Ventura County,1068,734,32.21,"43,553,733.57","POLYGON ((89734.878 -398866.128, 89801.849 -39...",25.00,2029fb1f894488,Upstream,0.00,"POLYGON ((92205.921 -401173.558, 92205.439 -40...",1.00,"31,365.48","31,365.48",0.00,"31,365.48","POLYGON ((92205.921 -401173.558, 92205.439 -40...",0.77,0.53,0.69,0.69,0.00,0.00,-1.12,-0.20,-0.20,118,118,118,118,11
22,60290033042,Kern County,1040,279,31.94,"1,051,991,808.27","POLYGON ((7651.644 -285527.714, 7653.631 -2855...",57.00,9ee5c520524407,Upstream,0.00,"POLYGON ((33162.910 -299372.866, 33162.428 -29...",1.00,"31,365.48","31,365.48",0.00,"274,294.75","MULTIPOLYGON (((47097.528 -304252.983, 47097.0...",0.27,0.07,0.27,0.27,0.00,0.01,0.58,-0.21,-0.30,118,118,118,118,11
45,60290045002,Kern County,1564,1448,70.23,"2,330,959,763.67","POLYGON ((-17549.472 -247500.198, -16929.899 -...",52.00,366b437a406177,Upstream,0.00,"POLYGON ((12118.019 -264999.261, 12117.537 -26...",1.00,"31,365.48","31,365.48",0.00,"94,096.45","MULTIPOLYGON (((27346.675 -273126.193, 27346.1...",0.06,0.06,0.93,0.93,0.00,0.00,-0.72,1.29,-0.34,118,118,118,118,11
44,60290045002,Kern County,1564,1448,70.23,"2,330,959,763.67","POLYGON ((-17549.472 -247500.198, -16929.899 -...",55.00,669bcab2913084,Upstream,0.00,"POLYGON ((27346.675 -273126.193, 27346.194 -27...",1.00,"31,365.48","31,365.48",0.00,"94,096.45","MULTIPOLYGON (((27346.675 -273126.193, 27346.1...",0.06,0.06,0.93,0.93,0.00,0.00,-0.72,1.29,-0.34,118,118,118,118,11
112,61110097003,Ventura County,733,62,3.13,"2,383,596,963.76","POLYGON ((50118.936 -399243.709, 50120.244 -39...",36.00,edb77f93276546,Upstream,0.00,"POLYGON ((101773.559 -392475.122, 101773.077 -...",1.00,"31,365.48","31,365.48",0.00,"58,775.47","POLYGON ((101719.575 -392563.846, 101711.827 -...",0.02,0.00,0.08,0.08,0.00,0.02,1.91,-1.34,-0.35,118,118,118,118,11
91,61110047041,Ventura County,1578,1239,45.85,"19,630,294.77","POLYGON ((78922.067 -427447.344, 78966.469 -42...",31.00,db471874509776,Upstream,0.00,"POLYGON ((80416.731 -428173.100, 80416.250 -42...",1.00,"31,365.48","31,365.48",0.00,"31,365.48","POLYGON ((80416.731 -428173.100, 80416.250 -42...",2.52,1.98,0.79,0.79,0.00,0.00,-1.12,0.34,0.14,118,118,118,118,11
56,60379800021,Los Angeles County,0,0,0.00,"6,977,891.51","POLYGON ((161372.391 -466647.635, 161408.314 -...",111.00,87ac6fd7167979,Refinery,0.00,"POLYGON ((163936.943 -465162.056, 163936.461 -...",1.00,"31,365.48","31

In [270]:
# There are a ton of duplicate rows getting introduced at some point of the process. Ideally I'll come back and 
# troubleshoot, but in the meantime I'm just removing these dupes. 
df_bg_intersect = df_bg_intersect.drop_duplicates(subset=['bg_id', 'flare_id'])

In [271]:
len(df_bg_intersect)

118

In [272]:
# # check calculations against two BGs
# # Should see a single "bcm_adj_tot" value per BG, and different adjusted "bcm_adj" values per flare_id 
# #depending on the amount of overlap.

# # filter the dataframe and format specified columns to display 4 decimal places
# filtered_df = df_bg_intersect_all[df_bg_intersect_all['bg_id'].isin([60379800331, 60379800111])].copy()
# filtered_df[['BCM_avg', f'{buffer_size}_bcm_int_adj', f'{buffer_size}_bcm_int_adj_tot', 
#              f'{buffer_size}_bcm_int_adj_tot_n']] = filtered_df[['BCM_avg', f'{buffer_size}_bcm_int_adj',
#                                                                  f'{buffer_size}_bcm_int_adj_tot', 
#                                                                  f'{buffer_size}_bcm_int_adj_tot_n']].apply(lambda x: x.apply(lambda y: '{:.4f}'.format(y)))


In [273]:
# filtered_df

### Split geom data from rest of data b/c column names must be <10 characters for shapefiles and that makes naming my different columns unnecessarily confusing. I export a csv of everything but the geometry, export a shapefile of just the geometry, then join the two back together in Tableau. 

In [274]:
df_bg_intersect.sample(1)

,bg_id,cnty_name,ACSTOTPOP,MINORPOP,D_PM25_2,bg_area,bg_geom,index_right,flare_id,flare_cate,BCM_avg,100_buff_geom,100_buff_int_prop,100_buff_int_area,100_buff_tot_area,100_bg_int_prop,100_bg_int_area,100_bg_int_geom,100_ACSTOTPOP_bg_int_count,100_MINORPOP_bg_int_count,MINORPOP_bg_totprop,100_bg_int_mnr_prop,100_bcm_int_adj,100_bcm_int_adj_tot,100_bcm_int_adj_tot_n,D_PM25_2_n,100_ACSTOTPOP_bg_int_count_n,100_bg_int_25%orless,100_bg_int_50%orless,100_bg_int_75%orless,100_bg_int_anyint,100_num_counties
21,60290033042,Kern County,1040,279,31.94,"1,051,991,808.27","POLYGON ((7651.644 -285527.714, 7653.631 -2855...",49.00,354ae1c9634624,Upstream,0.00,"POLYGON ((47065.526 -304151.318, 47065.044 -30...",1.00,"31,365.48","31,365.48",0.00,"274,294.75","MULTIPOLYGON (((47097.528 -304252.983, 47097.0...",0.27,0.07,0.27,0.27,0.00,0.01,0.58,-0.21,-0.30,118,118,118,118,11


### 100

In [275]:
df_bgintersect_geo100 = gp.GeoDataFrame(df_bg_intersect[['bg_id', 'flare_id', '100_bg_int_geom']], 
                                         geometry='100_bg_int_geom')

print(len(df_bgintersect_geo100[df_bgintersect_geo100.is_valid]))

df_bgintersect_100 = df_bg_intersect.drop(['bg_geom', f'{buffer_size}_buff_geom', f'{buffer_size}_bg_int_geom'], axis=1)

118


In [276]:
# Add in a buffer_size col for filtering in Tableau
df_bgintersect_geo100['buffer_size'] = 100

# and change the geom col name back to a general name so the 
# three buffer sizes can now be merged into one column
df_bgintersect_geo100.rename(columns={'100_bg_int_geom':'bg_int_geom'}, inplace=True)  # old:new. Match col names for merging

In [277]:
df_bgintersect_100.sample(2)

,bg_id,cnty_name,ACSTOTPOP,MINORPOP,D_PM25_2,bg_area,index_right,flare_id,flare_cate,BCM_avg,100_buff_int_prop,100_buff_int_area,100_buff_tot_area,100_bg_int_prop,100_bg_int_area,100_ACSTOTPOP_bg_int_count,100_MINORPOP_bg_int_count,MINORPOP_bg_totprop,100_bg_int_mnr_prop,100_bcm_int_adj,100_bcm_int_adj_tot,100_bcm_int_adj_tot_n,D_PM25_2_n,100_ACSTOTPOP_bg_int_count_n,100_bg_int_25%orless,100_bg_int_50%orless,100_bg_int_75%orless,100_bg_int_anyint,100_num_counties
112,61110097003,Ventura County,733,62,3.13,"2,383,596,963.76",36.00,edb77f93276546,Upstream,0.00,1.00,"31,365.48","31,365.48",0.00,"58,775.47",0.02,0.00,0.08,0.08,0.00,0.02,1.91,-1.34,-0.35,118,118,118,118,11
28,60290033042,Kern County,1040,279,31.94,"1,051,991,808.27",50.00,f43a9273287689,Upstream,0.00,1.00,"31,365.48","31,365.48",0.00,"274,294.75",0.27,0.07,0.27,0.27,0.00,0.01,0.58,-0.21,-0.30,118,118,118,118,11


In [278]:
df_bgintersect_geo100.sample(2)

,bg_id,flare_id,bg_int_geom,buffer_size
93,61110095004,2b7643a6398667,"MULTIPOLYGON (((62719.005 -410857.308, 62717.8...",100
30,60290033042,e97dd7e9596146,"MULTIPOLYGON (((47097.528 -304252.983, 47097.0...",100


In [ ]:
#df_bgintersect_geo100.to_file('data/df_bg_geo2000.shp', driver='ESRI Shapefile')

### 1000

In [254]:
df_bgintersect_geo1000 = gp.GeoDataFrame(df_bg_intersect[['bg_id', 'flare_id', '1000_bg_int_geom']], 
                                         geometry='1000_bg_int_geom')

print(len(df_bgintersect_geo1000[df_bgintersect_geo1000.is_valid]))

df_bgintersect_1000 = df_bg_intersect.drop(['bg_geom', f'{buffer_size}_buff_geom', 
                                            f'{buffer_size}_bg_int_geom'], axis=1)

# Add in a buffer_size col for filtering in Tableau
df_bgintersect_geo1000['buffer_size'] = 1000

# and change the geom col name back to a general name so the 
# three buffer sizes can now be merged into one column
df_bgintersect_geo1000.rename(columns={'1000_bg_int_geom':'bg_int_geom'}, inplace=True)  # old:new.

216


In [255]:
df_bgintersect_1000.head(2)

,bg_id,cnty_name,ACSTOTPOP,MINORPOP,D_PM25_2,bg_area,index_right,flare_id,flare_cate,BCM_avg,1000_buff_int_prop,1000_buff_int_area,1000_buff_tot_area,1000_bg_int_prop,1000_bg_int_area,1000_ACSTOTPOP_bg_int_count,1000_MINORPOP_bg_int_count,MINORPOP_bg_totprop,1000_bg_int_mnr_prop,1000_bcm_int_adj,1000_bcm_int_adj_tot,1000_bcm_int_adj_tot_n,D_PM25_2_n,1000_ACSTOTPOP_bg_int_count_n,1000_bg_int_25%orless,1000_bg_int_50%orless,1000_bg_int_75%orless,1000_bg_int_anyint,1000_num_counties
0,60133132032,Contra Costa County,2637,2125,47.88,"2,606,490.18",89.00,ba7a0d8d310483,Upstream,0.01,0.13,"418,998.49","3,136,548.49",0.16,"422,883.57",427.83,344.77,0.81,0.81,0.00,0.00,-0.30,0.31,0.95,193,204,213,216,11
1,60133132032,Contra Costa County,2637,2125,47.88,"2,606,490.18",90.00,6fb4852a406417,Upstream,0.01,0.13,"422,191.34","3,136,548.49",0.16,"422,883.57",427.83,344.77,0.81,0.81,0.00,0.00,-0.30,0.31,0.95,193,204,213,216,11


In [ ]:
#df_bgintersect_geo2000.to_file('data/df_bg_geo2000.shp', driver='ESRI Shapefile')

### 2000

In [233]:
# Create a new geodataframe with just the id and geom columns
df_bgintersect_geo2000 = gp.GeoDataFrame(df_bg_intersect[['bg_id', 'flare_id', '2000_bg_int_geom']], 
                                         geometry='2000_bg_int_geom')

print(len(df_bgintersect_geo2000[df_bgintersect_geo2000.is_valid]))


# Subset the original dataframe and drop unnecessary columns
df_bgintersect_2000 = df_bg_intersect.drop(['bg_geom', f'{buffer_size}_buff_geom', 
                                         f'{buffer_size}_bg_int_geom'], axis=1)

# Add in a buffer_size col for filtering in Tableau
df_bgintersect_geo2000['buffer_size'] = 2000

# and change the geom col name back to a general name so the 
# three buffer sizes can now be merged into one column
df_bgintersect_geo2000.rename(columns={'2000_bg_int_geom':'bg_int_geom'}, inplace=True)  # old:new.

501


In [234]:
df_bgintersect_2000.head(2)

,bg_id,cnty_name,ACSTOTPOP,MINORPOP,D_PM25_2,bg_area,index_right,flare_id,flare_cate,BCM_avg,2000_buff_int_prop,2000_buff_int_area,2000_buff_tot_area,2000_bg_int_prop,2000_bg_int_area,2000_ACSTOTPOP_bg_int_count,2000_MINORPOP_bg_int_count,MINORPOP_bg_totprop,2000_bg_int_mnr_prop,2000_bcm_int_adj,2000_bcm_int_adj_tot,2000_bcm_int_adj_tot_n,D_PM25_2_n,2000_ACSTOTPOP_bg_int_count_n,2000_bg_int_25%orless,2000_bg_int_50%orless,2000_bg_int_75%orless,2000_bg_int_anyint,2000_num_counties
0,60133132031,Contra Costa County,729,495,33.27,"514,270.79",89.00,ba7a0d8d310483,Upstream,0.01,0.04,"513,196.18","12,546,193.96",1.00,"513,196.18",727.48,493.97,0.68,0.68,0.00,0.00,-0.28,-0.50,0.19,239,311,365,501,11
1,60133132031,Contra Costa County,729,495,33.27,"514,270.79",90.00,6fb4852a406417,Upstream,0.01,0.04,"512,561.56","12,546,193.96",1.00,"513,196.18",727.48,493.97,0.68,0.68,0.00,0.00,-0.28,-0.50,0.19,239,311,365,501,11


In [ ]:
#df_bgintersect_geo5000.to_file('data/df_bg_geo5000.shp', driver='ESRI Shapefile')

### Combine

#### Social data

In [324]:
def filter_dataframe(df, prefix):
    cols_to_keep = ['bg_id', 'flare_id'] + [col for col in df.columns if col.startswith(prefix)]
    return df[cols_to_keep].copy()

In [325]:
# filter the 2nd and 3rd dataframes to only include columns starting with the buffer cols
# eg '5000_' plus the ID columns
# all the non "5000_" columns in these dfs are identical to the first df, so can be ignored.
# select columns that start with the appropriate prefix and keep 'bg_id' and 'flare_id'

df_bgintersect_100_filtered = filter_dataframe(df_bgintersect_100, '100_')
df_bgintersect_1000_filtered = filter_dataframe(df_bgintersect_1000, '1000_')

In [326]:
df_bg_intersect_temp = df_bgintersect_2000.merge(df_bgintersect_1000_filtered, 
                                               on=['bg_id', 'flare_id'], how='outer')


df_bg_intersect_all = df_bg_intersect_temp.merge(df_bgintersect_100_filtered, 
                                               on=['bg_id', 'flare_id'], how='outer')

In [327]:
len(df_bg_intersect_all)

501

In [328]:
df_bg_intersect_all.sample(1)

,bg_id,cnty_name,ACSTOTPOP,MINORPOP,D_PM25_2,bg_area,index_right,flare_id,flare_cate,BCM_avg,2000_buff_int_prop,2000_buff_int_area,2000_buff_tot_area,2000_bg_int_prop,2000_bg_int_area,2000_ACSTOTPOP_bg_int_count,2000_MINORPOP_bg_int_count,MINORPOP_bg_totprop,2000_bg_int_mnr_prop,2000_bcm_int_adj,2000_bcm_int_adj_tot,2000_bcm_int_adj_tot_n,D_PM25_2_n,2000_ACSTOTPOP_bg_int_count_n,2000_bg_int_25%orless,2000_bg_int_50%orless,2000_bg_int_75%orless,2000_bg_int_anyint,2000_num_counties,1000_buff_int_prop,1000_buff_int_area,1000_buff_tot_area,1000_bg_int_prop,1000_bg_int_area,1000_ACSTOTPOP_bg_int_count,1000_MINORPOP_bg_int_count,1000_bg_int_mnr_prop,1000_bcm_int_adj,1000_bcm_int_adj_tot,1000_bcm_int_adj_tot_n,1000_ACSTOTPOP_bg_int_count_n,1000_bg_int_25%orless,1000_bg_int_50%orless,1000_bg_int_75%orless,1000_bg_int_anyint,1000_num_counties,100_buff_int_prop,100_buff_int_area,100_buff_tot_area,100_bg_int_prop,100_bg_int_area,100_ACSTOTPOP_bg_int_count,100_MINORPOP_bg_int_count,100_bg_int_mnr_prop,100_bcm_int_adj,100_bcm_int_adj_tot,100_bcm_int_adj_tot_n,100_ACSTOTPOP_bg_int_count_n,100_bg_int_25%orless,100_bg_int_50%orless,100_bg_int_75%orless,100_bg_int_anyint,100_num_counties
63,60190082006,Fresno County,936,934,85.41,"421,645,619.21",75.00,96864b15606442,Upstream,0.00,1.00,"12,546,193.96","12,546,193.96",0.05,"22,236,272.77",49.36,49.26,1.00,1.00,0.00,0.00,-0.26,1.70,-0.76,239,311,365,501,11,1.00,"3,136,548.49","3,136,548.49",0.02,"6,397,212.80",14.20,14.17,1.00,0.00,0.00,-0.59,-0.49,193.00,204.00,213.00,216.00,11.00,1.00,"31,365.48","31,365.48",0.00,"94,096.45",0.21,0.21,1.00,0.00,0.00,-1.01,-0.31,118.00,118.00,118.00,118.00,11.00


In [335]:
df_bg_intersect_all[['BCM_avg', '2000_bcm_int_adj', '2000_bcm_int_adj_tot', '2000_bcm_int_adj_tot_n']] = df_bg_intersect_all[['BCM_avg', '2000_bcm_int_adj', '2000_bcm_int_adj_tot', '2000_bcm_int_adj_tot_n']].apply(lambda x: x.apply(lambda y: '{:.4f}'.format(y)))

In [336]:
df_bg_intersect_all.sample(2)

,bg_id,cnty_name,ACSTOTPOP,MINORPOP,D_PM25_2,bg_area,index_right,flare_id,flare_cate,BCM_avg,2000_buff_int_prop,2000_buff_int_area,2000_buff_tot_area,2000_bg_int_prop,2000_bg_int_area,2000_ACSTOTPOP_bg_int_count,2000_MINORPOP_bg_int_count,MINORPOP_bg_totprop,2000_bg_int_mnr_prop,2000_bcm_int_adj,2000_bcm_int_adj_tot,2000_bcm_int_adj_tot_n,D_PM25_2_n,2000_ACSTOTPOP_bg_int_count_n,2000_bg_int_25%orless,2000_bg_int_50%orless,2000_bg_int_75%orless,2000_bg_int_anyint,2000_num_counties,1000_buff_int_prop,1000_buff_int_area,1000_buff_tot_area,1000_bg_int_prop,1000_bg_int_area,1000_ACSTOTPOP_bg_int_count,1000_MINORPOP_bg_int_count,1000_bg_int_mnr_prop,1000_bcm_int_adj,1000_bcm_int_adj_tot,1000_bcm_int_adj_tot_n,1000_ACSTOTPOP_bg_int_count_n,1000_bg_int_25%orless,1000_bg_int_50%orless,1000_bg_int_75%orless,1000_bg_int_anyint,1000_num_counties,100_buff_int_prop,100_buff_int_area,100_buff_tot_area,100_bg_int_prop,100_bg_int_area,100_ACSTOTPOP_bg_int_count,100_MINORPOP_bg_int_count,100_bg_int_mnr_prop,100_bcm_int_adj,100_bcm_int_adj_tot,100_bcm_int_adj_tot_n,100_ACSTOTPOP_bg_int_count_n,100_bg_int_25%orless,100_bg_int_50%orless,100_bg_int_75%orless,100_bg_int_anyint,100_num_counties
192,60375758011,Los Angeles County,1518,1388,77.20,"389,881.95",106.00,771e0f67682313,Refinery,0.0011,0.03,"385,864.48","12,546,193.96",1.00,"389,881.95","1,518.00","1,388.00",0.91,0.91,0.0000,0.0001,-0.4418,1.35,1.30,239,311,365,501,11,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
387,60710125003,San Bernardino County,2054,2033,67.87,"1,105,798.32",115.00,c46ec700266912,Upstream,0.0000,0.06,"801,115.48","12,546,193.96",0.72,"801,115.48","1,488.06","1,472.84",0.99,0.99,0.0000,0.0000,-0.4740,0.96,1.25,239,311,365,501,11,0.02,"72,872.22","3,136,548.49",0.07,"72,872.22",135.36,133.97,0.99,0.00,0.00,-0.73,-0.07,193.00,204.00,213.00,216.00,11.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [337]:
#df_bg_intersect_all[df_bg_intersect_all['flare_id'] == '32334065611201']

df_bg_intersect_all[df_bg_intersect_all['bg_id'] == 60190086003]

,bg_id,cnty_name,ACSTOTPOP,MINORPOP,D_PM25_2,bg_area,index_right,flare_id,flare_cate,BCM_avg,2000_buff_int_prop,2000_buff_int_area,2000_buff_tot_area,2000_bg_int_prop,2000_bg_int_area,2000_ACSTOTPOP_bg_int_count,2000_MINORPOP_bg_int_count,MINORPOP_bg_totprop,2000_bg_int_mnr_prop,2000_bcm_int_adj,2000_bcm_int_adj_tot,2000_bcm_int_adj_tot_n,D_PM25_2_n,2000_ACSTOTPOP_bg_int_count_n,2000_bg_int_25%orless,2000_bg_int_50%orless,2000_bg_int_75%orless,2000_bg_int_anyint,2000_num_counties,1000_buff_int_prop,1000_buff_int_area,1000_buff_tot_area,1000_bg_int_prop,1000_bg_int_area,1000_ACSTOTPOP_bg_int_count,1000_MINORPOP_bg_int_count,1000_bg_int_mnr_prop,1000_bcm_int_adj,1000_bcm_int_adj_tot,1000_bcm_int_adj_tot_n,1000_ACSTOTPOP_bg_int_count_n,1000_bg_int_25%orless,1000_bg_int_50%orless,1000_bg_int_75%orless,1000_bg_int_anyint,1000_num_counties,100_buff_int_prop,100_buff_int_area,100_buff_tot_area,100_bg_int_prop,100_bg_int_area,100_ACSTOTPOP_bg_int_count,100_MINORPOP_bg_int_count,100_bg_int_mnr_prop,100_bcm_int_adj,100_bcm_int_adj_tot,100_bcm_int_adj_tot_n,100_ACSTOTPOP_bg_int_count_n,100_bg_int_25%orless,100_bg_int_50%orless,100_bg_int_75%orless,100_bg_int_anyint,100_num_counties
67,60190086003,Fresno County,763,620,59.88,"40,739,634.58",80.00,4482a7f6283749,Upstream,0.0001,0.84,"10,519,605.06","12,546,193.96",0.26,"10,775,008.06",201.80,163.98,0.81,0.81,0.0001,0.0035,0.4942,0.62,-0.55,239,311,365,501,11,0.91,"2,856,574.50","3,136,548.49",0.07,"2,989,772.71",55.99,45.50,0.81,0.00,0.00,-0.02,-0.34,193.00,204.00,213.00,216.00,11.00,1.00,"31,365.48","31,365.48",0.00,"46,858.93",0.88,0.71,0.81,0.00,0.00,-0.49,-0.18,118.00,118.00,118.00,118.00,11.00
68,60190086003,Fresno County,763,620,59.88,"40,739,634.58",79.00,c8499b30141447,Upstream,0.0026,0.84,"10,491,607.39","12,546,193.96",0.26,"10,775,008.06",201.80,163.98,0.81,0.81,0.0022,0.0035,0.4942,0.62,-0.55,239,311,365,501,11,0.91,"2,842,304.89","3,136,548.49",0.07,"2,989,772.71",55.99,45.50,0.81,0.00,0.00,-0.02,-0.34,193.00,204.00,213.00,216.00,11.00,1.00,"31,365.48","31,365.48",0.00,"46,858.93",0.88,0.71,0.81,0.00,0.00,-0.49,-0.18,118.00,118.00,118.00,118.00,11.00
69,60190086003,Fresno County,763,620,59.88,"40,739,634.58",76.00,1dc02697717831,Upstream,0.0016,0.83,"10,473,400.02","12,546,193.96",0.26,"10,775,008.06",201.80,163.98,0.81,0.81,0.0013,0.0035,0.4942,0.62,-0.55,239,311,365,501,11,0.90,"2,834,506.88","3,136,548.49",0.07,"2,989,772.71",55.99,45.50,0.81,0.00,0.00,-0.02,-0.34,193.00,204.00,213.00,216.00,11.00,1.00,"31,365.48","31,365.48",0.00,"46,858.93",0.88,0.71,0.81,0.00,0.00,-0.49,-0.18,118.00,118.00,118.00,118.00,11.00


In [315]:
df_bg_intersect_all.to_csv('data/df_bg_impact_all.tsv', sep='\t', index=False)

#### Geo data

In [316]:
df_bg_intersect_geoall = pd.concat([df_bgintersect_geo100, 
                                    df_bgintersect_geo1000,
                                    df_bgintersect_geo2000])

In [318]:
# num intersections at 2000m: 501
# at 1000m: 216
# at 100m: 118
# Should equal 835 total
len(df_bg_intersect_geoall)

835

In [287]:
#df_bg_intersect_geoall.sample(10)

In [319]:
df_bg_intersect_geoall = df_bg_intersect_geoall.set_geometry('bg_int_geom')
df_bg_intersect_geoall.to_file('data/df_bg_geoall.shp', driver='ESRI Shapefile')

/Users/natebender/.local/lib/python3.7/site-packages/ipykernel_launcher.py:2: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  


# Aggregate by Flare ID and export files for Tableau

Take baseline dfs and create copies clearly defined for this flare buffer analysis, 
separate from the work that is needed for the BG buffer analysis above. 
Both analyses modify these same two dfs in different ways; need to ensure I keep the calculations separated.
Plus, they'll go into Tableau as different dataframes anyway. 

In [186]:
# Use the baseline df as the starting point

df_baseline_all.sample(1)

,bg_id,cnty_name,ACSTOTPOP,MINORPOP,D_PM25_2,bg_area,bg_geom,index_right,flare_id,flare_cate,BCM_avg,2000_buff_geom,2000_buff_int_prop,2000_buff_int_area,2000_buff_tot_area,2000_bg_int_prop,2000_bg_int_area,2000_bg_int_geom,2000_ACSTOTPOP_bg_int_count,2000_MINORPOP_bg_int_count,MINORPOP_bg_totprop,2000_bg_int_mnr_prop
1937,60590626551,Orange County,731,641,40.34,"1,264,270.59","POLYGON ((200855.377 -484532.202, 200900.565 -...",87.00,9a8711bd211546,Upstream,0.00,"POLYGON ((204140.769 -486732.803, 204131.138 -...",0.06,"742,934.50","12,546,193.96",0.59,"742,934.50","POLYGON ((202811.773 -484851.280, 202809.668 -...",429.56,376.68,0.88,0.88


In [187]:
# Define the variables to aggregate

# BCM_avg: No adjustments needed because bcm is already attached to the flare unit of analysis
# D_PM25_D: Create new var of avg PM25 score for all BGs in each buffer 
# ACSTOTPOP_bg_int_count: Create new var of total population in all intersections in each buffer

agg_dict = {'D_PM25_2': 'mean', f'{buffer_size}_ACSTOTPOP_bg_int_count': 'sum', f'{buffer_size}_MINORPOP_bg_int_count': 'sum'}

# set up df for flare-specific calcs
flare_intersect = df_baseline_all.copy()

# Group the block groups by flare ID and aggregrate the selected vars
flare_agg = df_baseline_all.groupby('flare_id').agg(agg_dict)

# Rename the output variables
flare_agg = flare_agg.rename(columns={'D_PM25_2': f'{buffer_size}_int_pm25_avg', 
                                      f'{buffer_size}_ACSTOTPOP_bg_int_count': f'{buffer_size}_ACSTOTPOP_bg_int_sum',
                                     f'{buffer_size}_MINORPOP_bg_int_count': f'{buffer_size}_MINORPOP_bg_int_sum'})

# Join the aggregated variables with the original columns
flare_intersect = flare_intersect.merge(flare_agg, on='flare_id')

# add a new column for proportion of minorities within the given buffer zone
flare_intersect[f'{buffer_size}_flare_minor_prop'] = (flare_intersect[f'{buffer_size}_MINORPOP_bg_int_sum'] / 
                                                      flare_intersect[f'{buffer_size}_ACSTOTPOP_bg_int_sum'])


In [188]:
flare_intersect.sample(3)

,bg_id,cnty_name,ACSTOTPOP,MINORPOP,D_PM25_2,bg_area,bg_geom,index_right,flare_id,flare_cate,BCM_avg,2000_buff_geom,2000_buff_int_prop,2000_buff_int_area,2000_buff_tot_area,2000_bg_int_prop,2000_bg_int_area,2000_bg_int_geom,2000_ACSTOTPOP_bg_int_count,2000_MINORPOP_bg_int_count,MINORPOP_bg_totprop,2000_bg_int_mnr_prop,2000_int_pm25_avg,2000_ACSTOTPOP_bg_int_sum,2000_MINORPOP_bg_int_sum,2000_flare_minor_prop
206,60590626435,Orange County,709,56,7.01,"891,214.00","POLYGON ((201504.089 -487843.108, 201647.481 -...",86.00,f86bc4a0328919,Upstream,0.01,"POLYGON ((204129.924 -486745.806, 204120.294 -...",0.07,"891,214.00","12,546,193.96",1.00,"891,214.00","POLYGON ((201504.089 -487843.108, 201647.481 -...",709.00,56.00,0.08,0.08,25.34,"7,614.00","3,092.89",0.41
89,60375759021,Los Angeles County,1899,921,39.82,"165,995.87","POLYGON ((167222.605 -469968.550, 167221.306 -...",108.00,b91dbc75798361,Refinery,0.00,"POLYGON ((167283.806 -469521.668, 167274.175 -...",0.00,"23,009.09","12,546,193.96",0.14,"23,009.09","POLYGON ((167222.605 -469968.550, 167221.306 -...",263.23,127.66,0.48,0.48,66.74,"19,891.22","16,554.81",0.83
442,60133200011,Contra Costa County,1203,434,32.95,"14,699,356.10","POLYGON ((-186720.000 3934.869, -186718.573 39...",96.00,504fcec7336969,Refinery,0.00,"POLYGON ((-178447.932 3117.456, -178457.562 29...",0.11,"1,400,711.31","12,546,193.96",0.84,"12,380,008.32","POLYGON ((-185794.763 4518.387, -185742.949 45...","1,013.18",365.52,0.36,0.36,30.56,"1,371.26",463.90,0.34


In [189]:
# Define the variables to standardize

varlist = ['BCM_avg', f'{buffer_size}_int_pm25_avg', f'{buffer_size}_ACSTOTPOP_bg_int_sum']  

# Create a StandardScaler object that will transform selected variables to have a mean of zero and 
# sd of 1.
scaler = StandardScaler()

# Fit and transform the selected variables
intersect_norm = scaler.fit_transform(flare_intersect[varlist])

# Create new variables with the standardized values
intersect_norm = pd.DataFrame(intersect_norm, columns=[var + '_n' for var in varlist])

# Concatenate the new variables with the original DataFrame
flare_intersect = pd.concat([flare_intersect, intersect_norm], axis=1)

In [190]:
flare_intersect.sample(2)

,bg_id,cnty_name,ACSTOTPOP,MINORPOP,D_PM25_2,bg_area,bg_geom,index_right,flare_id,flare_cate,BCM_avg,2000_buff_geom,2000_buff_int_prop,2000_buff_int_area,2000_buff_tot_area,2000_bg_int_prop,2000_bg_int_area,2000_bg_int_geom,2000_ACSTOTPOP_bg_int_count,2000_MINORPOP_bg_int_count,MINORPOP_bg_totprop,2000_bg_int_mnr_prop,2000_int_pm25_avg,2000_ACSTOTPOP_bg_int_sum,2000_MINORPOP_bg_int_sum,2000_flare_minor_prop,BCM_avg_n,2000_int_pm25_avg_n,2000_ACSTOTPOP_bg_int_sum_n
188,60374034031,Los Angeles County,2523,2452,59.28,"1,651,077.09","POLYGON ((199236.773 -439529.661, 199285.681 -...",104.00,a8689599458336,Upstream,0.00,"POLYGON ((203158.098 -439404.603, 203148.468 -...",0.13,"1,651,077.09","12,546,193.96",1.00,"1,651,077.09","POLYGON ((199236.773 -439529.661, 199285.681 -...","2,523.00","2,452.00",0.97,0.97,47.79,"12,496.66","9,956.11",0.80,-0.50,0.14,0.22
301,61110085022,Ventura County,2256,492,14.19,"44,758,496.21","POLYGON ((109297.102 -413012.239, 109298.039 -...",34.00,62e277c4111380,Upstream,0.00,"POLYGON ((113639.854 -403970.195, 113630.224 -...",0.01,"86,460.52","12,546,193.96",0.01,"226,450.47","POLYGON ((113375.209 -405189.253, 113375.885 -...",11.41,2.49,0.22,0.22,16.22,192.73,115.39,0.60,-0.28,-1.43,-0.98


### 100

In [162]:
# Create a new geodataframe with just the id and geom columns
# flare_intersect_geo = flare_intersect[['flare_id', 'buff2000_geom']].copy()
flareintersect_geo100 = gp.GeoDataFrame(flare_intersect[['flare_id', '100_buff_geom']].copy(), 
                                      geometry='100_buff_geom', crs=meters_crs)


# Subset the original dataframe and drop unnecessary columns
flareintersect_100 = flare_intersect.drop(['bg_geom', '100_bg_int_geom', '100_buff_geom'], axis=1)

# Add in a buffer_size col for filtering in Tableau
flareintersect_geo100['buffer_size'] = 100

# and change the geom col name back to a general name so the 
# three buffer sizes can now be merged into one column
flareintersect_geo100.rename(columns={'100_buff_geom':'buff_geom'}, inplace=True)  # old:new.

In [163]:
flareintersect_100.sample(1)

,bg_id,cnty_name,ACSTOTPOP,MINORPOP,D_PM25_2,bg_area,index_right,flare_id,flare_cate,BCM_avg,100_buff_int_prop,100_buff_int_area,100_buff_tot_area,100_bg_int_prop,100_bg_int_area,100_ACSTOTPOP_bg_int_count,100_MINORPOP_bg_int_count,MINORPOP_bg_totprop,100_bg_int_mnr_prop,100_int_pm25_avg,100_ACSTOTPOP_bg_int_sum,100_MINORPOP_bg_int_sum,100_flare_minor_prop,BCM_avg_n,100_int_pm25_avg_n,100_ACSTOTPOP_bg_int_sum_n
13,60290045002,Kern County,1564,1448,70.23,"2,330,959,763.67",46.00,dc3b0188369130,Upstream,0.00,1.00,"31,365.48","31,365.48",0.00,"94,096.45",0.06,0.06,0.93,0.93,70.23,0.06,0.06,0.93,-0.60,1.29,-0.30


### 1000

In [176]:
# Create a new geodataframe with just the id and geom columns
# flare_intersect_geo = flare_intersect[['flare_id', 'buff2000_geom']].copy()
flareintersect_geo1000 = gp.GeoDataFrame(flare_intersect[['flare_id', '1000_buff_geom']].copy(), 
                                      geometry='1000_buff_geom', crs=meters_crs)


# Subset the original dataframe and drop unnecessary columns
flareintersect_1000 = flare_intersect.drop(['bg_geom', '1000_bg_int_geom', '1000_buff_geom'], axis=1)

# Add in a buffer_size col for filtering in Tableau
flareintersect_geo1000['buffer_size'] = 1000

# and change the geom col name back to a general name so the 
# three buffer sizes can now be merged into one column
flareintersect_geo1000.rename(columns={'1000_buff_geom':'buff_geom'}, inplace=True)  # old:new.

In [177]:
flareintersect_1000.sample(2)

,bg_id,cnty_name,ACSTOTPOP,MINORPOP,D_PM25_2,bg_area,index_right,flare_id,flare_cate,BCM_avg,1000_buff_int_prop,1000_buff_int_area,1000_buff_tot_area,1000_bg_int_prop,1000_bg_int_area,1000_ACSTOTPOP_bg_int_count,1000_MINORPOP_bg_int_count,MINORPOP_bg_totprop,1000_bg_int_mnr_prop,1000_int_pm25_avg,1000_ACSTOTPOP_bg_int_sum,1000_MINORPOP_bg_int_sum,1000_flare_minor_prop,BCM_avg_n,1000_int_pm25_avg_n,1000_ACSTOTPOP_bg_int_sum_n
194,60190079032,Fresno County,4735,3754,28.51,"1,436,574,980.80",81.00,74ebeb86781258,Upstream,0.00,0.37,"1,157,684.73","3,136,548.49",0.00,"1,157,684.73",3.82,3.03,0.79,0.79,56.86,14.70,13.76,0.94,-0.53,0.72,-0.61
149,60133150001,Contra Costa County,885,209,25.20,"38,922,697.19",92.00,ec71d30c876528,Refinery,0.00,1.00,"3,136,548.49","3,136,548.49",0.09,"3,657,463.67",83.16,19.64,0.24,0.24,25.20,83.16,19.64,0.24,0.26,-0.66,-0.56


### 2000

In [191]:
# Create a new geodataframe with just the id and geom columns
# flare_intersect_geo = flare_intersect[['flare_id', 'buff2000_geom']].copy()
flareintersect_geo2000 = gp.GeoDataFrame(flare_intersect[['flare_id', '2000_buff_geom']].copy(), 
                                      geometry='2000_buff_geom', crs=meters_crs)


# Subset the original dataframe and drop unnecessary columns
flareintersect_2000 = flare_intersect.drop(['bg_geom', '2000_bg_int_geom', '2000_buff_geom'], axis=1)

# Add in a buffer_size col for filtering in Tableau
flareintersect_geo2000['buffer_size'] = 2000

# and change the geom col name back to a general name so the 
# three buffer sizes can now be merged into one column
flareintersect_geo2000.rename(columns={'2000_buff_geom':'buff_geom'}, inplace=True)  # old:new.

In [192]:
flareintersect_2000.sample(1)

,bg_id,cnty_name,ACSTOTPOP,MINORPOP,D_PM25_2,bg_area,index_right,flare_id,flare_cate,BCM_avg,2000_buff_int_prop,2000_buff_int_area,2000_buff_tot_area,2000_bg_int_prop,2000_bg_int_area,2000_ACSTOTPOP_bg_int_count,2000_MINORPOP_bg_int_count,MINORPOP_bg_totprop,2000_bg_int_mnr_prop,2000_int_pm25_avg,2000_ACSTOTPOP_bg_int_sum,2000_MINORPOP_bg_int_sum,2000_flare_minor_prop,BCM_avg_n,2000_int_pm25_avg_n,2000_ACSTOTPOP_bg_int_sum_n
265,61110023022,Ventura County,756,394,19.56,"2,160,148.14",19.00,d88cbfaf112227,Upstream,0.00,0.09,"1,095,623.27","12,546,193.96",0.53,"1,144,303.34",400.48,208.72,0.52,0.52,19.62,"2,405.14","1,310.22",0.54,0.48,-1.27,-0.77


### Combine

#### Geo data

In [193]:
flareintersect_geo100.sample(1)

,flare_id,buff_geom,buffer_size
76,32334065611201,"POLYGON ((22554.334 -149318.635, 22553.853 -14...",100


In [194]:
flareintersect_geo1000.sample(1)

,flare_id,buff_geom,buffer_size
135,ebeeb5ca215591,"POLYGON ((70737.988 -420210.310, 70733.173 -42...",1000


In [195]:
flareintersect_geo2000.sample(1)

,flare_id,buff_geom,buffer_size
270,4985ed9a989022,"POLYGON ((69527.269 -411223.881, 69517.638 -41...",2000


In [196]:
df_flareintersect_geoall = pd.concat([flareintersect_geo100, 
                                    flareintersect_geo1000,
                                    flareintersect_geo2000])

In [197]:
df_flareintersect_geoall.sample(10)

,flare_id,buff_geom,buffer_size
44,ba7a0d8d310483,"POLYGON ((-169488.809 339.717, -169489.291 329...",100
32,b0c5ecc2782755,"POLYGON ((203125.440 -486756.777, 203120.625 -...",1000
62,68580388957987,"POLYGON ((167134.921 -469587.135, 167125.291 -...",2000
259,63ff9323420679,"POLYGON ((69346.315 -411230.717, 69336.684 -41...",2000
434,6fb4852a406417,"POLYGON ((-167598.970 344.705, -167608.600 148...",2000
471,bb3774e2262896,"POLYGON ((103661.023 -392392.474, 103651.393 -...",2000
70,c46ec700266912,"POLYGON ((246587.514 -437618.082, 246582.699 -...",1000
60,6ab710b4231001,"POLYGON ((91633.000 -407577.668, 91628.184 -40...",1000
404,d66c940d551612,"POLYGON ((-208209.930 -5388.237, -208219.561 -...",2000
330,3f1a890b678530,"POLYGON ((-182049.829 3544.046, -182059.460 33...",2000


In [198]:
df_flareintersect_geoall = df_flareintersect_geoall.set_geometry('buff_geom')
df_flareintersect_geoall.to_file('data/df_flare_geoall.shp', driver='ESRI Shapefile')

/Users/natebender/.local/lib/python3.7/site-packages/ipykernel_launcher.py:2: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  


#### Social data

In [199]:
# Keep the two relevant ID columns plus the prefix for the buffer-specific columns for each buffer size

def filter_dataframe(df, prefix):
    cols_to_keep = ['bg_id', 'flare_id'] + [col for col in df.columns if col.startswith(prefix)]
    return df[cols_to_keep].copy()

In [200]:
# filter the 2nd and 3rd dataframes to only include columns starting with the buffer cols
# eg '5000_' plus the ID columns
# all the non "5000_" columns in these dfs are identical to the first df, so can be ignored.
# select columns that start with the appropriate prefix and keep 'bg_id' and 'flare_id'

flareintersect_100_filtered = filter_dataframe(flareintersect_100, '100_')
flareintersect_1000_filtered = filter_dataframe(flareintersect_1000, '1000_')

In [201]:
df_flare_intersect_temp = flareintersect_2000.merge(flareintersect_1000_filtered, 
                                               on=['bg_id', 'flare_id'], how='outer')


df_flare_intersect_all = df_flare_intersect_temp.merge(flareintersect_100_filtered, 
                                               on=['bg_id', 'flare_id'], how='outer')

In [213]:
df_flare_intersect_all[(df_flare_intersect_all['flare_id'] == '32334065611201')]

,bg_id,cnty_name,ACSTOTPOP,MINORPOP,D_PM25_2,bg_area,index_right,flare_id,flare_cate,BCM_avg,2000_buff_int_prop,2000_buff_int_area,2000_buff_tot_area,2000_bg_int_prop,2000_bg_int_area,2000_ACSTOTPOP_bg_int_count,2000_MINORPOP_bg_int_count,MINORPOP_bg_totprop,2000_bg_int_mnr_prop,2000_int_pm25_avg,2000_ACSTOTPOP_bg_int_sum,2000_MINORPOP_bg_int_sum,2000_flare_minor_prop,BCM_avg_n,2000_int_pm25_avg_n,2000_ACSTOTPOP_bg_int_sum_n,1000_buff_int_prop,1000_buff_int_area,1000_buff_tot_area,1000_bg_int_prop,1000_bg_int_area,1000_ACSTOTPOP_bg_int_count,1000_MINORPOP_bg_int_count,1000_bg_int_mnr_prop,1000_int_pm25_avg,1000_ACSTOTPOP_bg_int_sum,1000_MINORPOP_bg_int_sum,1000_flare_minor_prop,1000_int_pm25_avg_n,1000_ACSTOTPOP_bg_int_sum_n,100_buff_int_prop,100_buff_int_area,100_buff_tot_area,100_bg_int_prop,100_bg_int_area,100_ACSTOTPOP_bg_int_count,100_MINORPOP_bg_int_count,100_bg_int_mnr_prop,100_int_pm25_avg,100_ACSTOTPOP_bg_int_sum,100_MINORPOP_bg_int_sum,100_flare_minor_prop,100_int_pm25_avg_n,100_ACSTOTPOP_bg_int_sum_n
356,60190012022,Fresno County,3169,3138,81.72,"2,750,945.12",85.00,32334065611201,Refinery,0.00,0.04,"474,083.04","12,546,193.96",0.17,"474,083.04",546.13,540.79,0.99,0.99,71.16,"1,327.34","1,191.38",0.90,-0.48,1.30,-0.87,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
357,60190015002,Fresno County,1634,1388,68.82,"26,452,825.51",85.00,32334065611201,Refinery,0.00,0.16,"2,038,053.42","12,546,193.96",0.08,"2,038,053.42",125.89,106.94,0.85,0.85,71.16,"1,327.34","1,191.38",0.90,-0.48,1.30,-0.87,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
358,60190015001,Fresno County,703,586,73.44,"11,778,713.09",85.00,32334065611201,Refinery,0.00,0.62,"7,733,788.66","12,546,193.96",0.66,"7,733,788.66",461.58,384.76,0.83,0.83,71.16,"1,327.34","1,191.38",0.90,-0.48,1.30,-0.87,1.00,"3,136,548.49","3,136,548.49",0.27,"3,136,548.49",187.20,156.05,0.83,73.44,187.20,156.05,0.83,1.45,-0.47,1.00,"31,365.48","31,365.48",0.00,"31,365.48",1.87,1.56,0.83,73.44,1.87,1.56,0.83,1.42,-0.06
359,60190017001,Fresno County,1718,1409,60.65,"20,398,062.79",85.00,32334065611201,Refinery,0.00,0.18,"2,300,268.84","12,546,193.96",0.11,"2,300,268.84",193.74,158.89,0.82,0.82,71.16,"1,327.34","1,191.38",0.90,-0.48,1.30,-0.87,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [290]:
# Replace NA with 0 for all the buffer-specific columns

cols = df_flare_intersect_all.columns[df_flare_intersect_all.columns.str.startswith(('100_', '1000_', '2000_'))]
df_flare_intersect_all[cols] = df_flare_intersect_all[cols].fillna(0)


In [291]:
df_flare_intersect_all[(df_flare_intersect_all['flare_id'] == '32334065611201')]

,bg_id,cnty_name,ACSTOTPOP,MINORPOP,D_PM25_2,bg_area,index_right,flare_id,flare_cate,BCM_avg,2000_buff_int_prop,2000_buff_int_area,2000_buff_tot_area,2000_bg_int_prop,2000_bg_int_area,2000_ACSTOTPOP_bg_int_count,2000_MINORPOP_bg_int_count,MINORPOP_bg_totprop,2000_bg_int_mnr_prop,2000_int_pm25_avg,2000_ACSTOTPOP_bg_int_sum,2000_MINORPOP_bg_int_sum,2000_flare_minor_prop,BCM_avg_n,2000_int_pm25_avg_n,2000_ACSTOTPOP_bg_int_sum_n,1000_buff_int_prop,1000_buff_int_area,1000_buff_tot_area,1000_bg_int_prop,1000_bg_int_area,1000_ACSTOTPOP_bg_int_count,1000_MINORPOP_bg_int_count,1000_bg_int_mnr_prop,1000_int_pm25_avg,1000_ACSTOTPOP_bg_int_sum,1000_MINORPOP_bg_int_sum,1000_flare_minor_prop,1000_int_pm25_avg_n,1000_ACSTOTPOP_bg_int_sum_n,100_buff_int_prop,100_buff_int_area,100_buff_tot_area,100_bg_int_prop,100_bg_int_area,100_ACSTOTPOP_bg_int_count,100_MINORPOP_bg_int_count,100_bg_int_mnr_prop,100_int_pm25_avg,100_ACSTOTPOP_bg_int_sum,100_MINORPOP_bg_int_sum,100_flare_minor_prop,100_int_pm25_avg_n,100_ACSTOTPOP_bg_int_sum_n
356,60190012022,Fresno County,3169,3138,81.72,"2,750,945.12",85.00,32334065611201,Refinery,0.00,0.04,"474,083.04","12,546,193.96",0.17,"474,083.04",546.13,540.79,0.99,0.99,71.16,"1,327.34","1,191.38",0.90,-0.48,1.30,-0.87,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
357,60190015002,Fresno County,1634,1388,68.82,"26,452,825.51",85.00,32334065611201,Refinery,0.00,0.16,"2,038,053.42","12,546,193.96",0.08,"2,038,053.42",125.89,106.94,0.85,0.85,71.16,"1,327.34","1,191.38",0.90,-0.48,1.30,-0.87,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
358,60190015001,Fresno County,703,586,73.44,"11,778,713.09",85.00,32334065611201,Refinery,0.00,0.62,"7,733,788.66","12,546,193.96",0.66,"7,733,788.66",461.58,384.76,0.83,0.83,71.16,"1,327.34","1,191.38",0.90,-0.48,1.30,-0.87,1.00,"3,136,548.49","3,136,548.49",0.27,"3,136,548.49",187.20,156.05,0.83,73.44,187.20,156.05,0.83,1.45,-0.47,1.00,"31,365.48","31,365.48",0.00,"31,365.48",1.87,1.56,0.83,73.44,1.87,1.56,0.83,1.42,-0.06
359,60190017001,Fresno County,1718,1409,60.65,"20,398,062.79",85.00,32334065611201,Refinery,0.00,0.18,"2,300,268.84","12,546,193.96",0.11,"2,300,268.84",193.74,158.89,0.82,0.82,71.16,"1,327.34","1,191.38",0.90,-0.48,1.30,-0.87,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00


In [292]:
df_flare_intersect_all.to_csv('data/df_flare_impact_all.tsv', sep='\t', index=False)

### Pivot CaFlares df for Tableau

In [212]:
ca_flares.sample(1)

,flare_id,flare_cate,cnty_name,BCM_avg,flarepts_geom,100_buff_geom,1000_buff_geom,2000_buff_geom
97,a4ccab56112862,Refinery,Contra Costa County,0.00,POINT (-180758.535 2967.647),"POLYGON ((-180658.535 2967.647, -180659.016 29...","POLYGON ((-179758.535 2967.647, -179763.350 28...","POLYGON ((-178758.535 2967.647, -178768.165 27..."


In [208]:
for_melt.sample(1)

,Country,ISO Code,Catalog ID,id #,Latitude,Longitude,BCM_2019,avg_temp,Ellipticit,Detection_,clr_obs19,Type,ID 2020,BCM_2020,Avg. temp.,Det_freq20,clr_obs20,ID 2021,BCM_2021,Det_freq21,clr_obs21,id_key_201,Avg_Temp_K,Det_freq17,clr_obs17,ISO_Code,BCM_2017,id_key,BCM_2012,BCM_2013,BCM_2014,BCM_2015,BCM_2016,clr_obs12,clr_obs13,clr_obs14,clr_obs15,clr_obs16,Det_freq12,Det_freq13,Det_freq14,Det_freq15,Det_freq16,Catalog _1,BCM_2018,Det_freq18,clr_obs18,flare_cate,Detection,Det_freq19,flare_id,STATEFP,COUNTYFP,COUNTYNS,GEOID,NAME,cnty_name,LSAD,CLASSFP,MTFCC,CSAFP,CBSAFP,METDIVFP,FUNCSTAT,ALAND,AWATER,INTPTLAT,INTPTLON,BCM_avg,geometry
115,United States,None,None,NaN,34.05,-117.34,NaN,NaN,1.60,NaN,NaN,upstream,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"1,810.00",NaN,NaN,USA,NaN,VNF_e2016_n1890_x1173398W_y340460N_v1.0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,408.00,0.00,0.00,0.00,0.00,0.98,None,NaN,NaN,NaN,Upstream,None,None,c46ec700266912,06,071,00277300,06071,San Bernardino,San Bernardino County,06,H1,G4020,348,40140,None,A,51948120575,123845026,+34.8572198,-116.1811967,0.00,POINT (245587.514 -437618.082)


In [209]:
for_melt = gp.read_file("data/ca_flares_clean.shp") 

for_melt = for_melt[['flare_id', 'BCM_2012', 'BCM_2013', 'BCM_2014', 'BCM_2015', 'BCM_2016', 'BCM_2017', 'BCM_2018', 'BCM_2019', 'BCM_2020', 'BCM_2021']].copy()

value_vars = ['BCM_2012', 'BCM_2013', 'BCM_2014', 'BCM_2015', 'BCM_2016', 'BCM_2017',
              'BCM_2018', 'BCM_2019', 'BCM_2020', 'BCM_2021']

ca_flares_melt = pd.melt(for_melt, 
                    id_vars=[col for col in for_melt.columns if col not in value_vars],
                    value_vars=value_vars, var_name='year', value_name='bcm_values')

# extract the year from the column name
ca_flares_melt['year'] = ca_flares_melt['year'].str.extract('(\d+)').astype(int)

In [210]:
ca_flares_melt.sort_values('bcm_values', ascending=False)

,flare_id,year,bcm_values
1022,f86bc4a0328919,2020,0.01
52,366b437a406177,2012,0.01
1025,ba7a0d8d310483,2020,0.01
1143,6fb4852a406417,2021,0.01
177,bba40bf5222151,2013,0.01
...,...,...,...
1165,b5b74874510417,2021,NaN
1166,915ae204194312,2021,NaN
1167,e3ff8fc8166258,2021,NaN
1168,c46ec700266912,2021,NaN


In [211]:
ca_flares_melt.to_csv('data/df_flare_bcmpivot.tsv', sep='\t', index=False)